In [55]:
import numpy as np
import matplotlib.pyplot as plt

In [56]:
DB = {} #database
B = 20 #MHz; bandwidth of each block

M = 10 #number of f-blocks
density = 125e-6
N = 25 #number of nodes

area = N/density
L = np.sqrt(area)

epsilon = -1e-5
alpha = 2.5 #pathloss exponent
noise = 1e-5 #W; Rx noise
P = 1 #W; Tx power of each access point
R = 50.0 #m; range of an access point
N_ITER = 1000
turn_count = 100

d_max = 300 #m; max distance between two neighbors

In [57]:
class Node:
    def __init__(self, x, y, ID):
        self.x = x
        self.y = y
        self.ID = str(ID)
        self.neighbors = {}
        self.neighbor_nodes = []
        self.BSI = None
        self.datarate_min = None

    def init_BSI(self):
        self.BSI = np.ones(M, dtype='float64')

    def publish_BSI(self, DB):
        DB[self.ID] = self.BSI

    def add_neighbor(self, ID, d):
        self.neighbors[str(ID)] = d
        self.neighbor_nodes.append(ID)

    def update_BSI(self, N, DB, node_list):
        if len(self.neighbor_nodes)>0:
            # collect the BSI of neighbors and scale them by 1/distance^2
            score = np.zeros(M, dtype='float64')
            for nn in self.neighbor_nodes:
                # suggestion from the neighbours
                this_BSI = -np.power(max(1,self.neighbors[str(nn)]), -alpha)*DB[str(nn)]
                score = score + this_BSI

            # update the BSI by occupying the positive scores
            not_chosen = []

            # social learning
            for m in range(M):
                if score[m] > epsilon:
                    self.BSI[m] = 1.0
                else:
                    self.BSI[m] = -1.0
                    not_chosen.append(m)

            # check if the datarate requirements are satisfied:
            # by calculating the performance in each f-block
            self.publish_BSI(DB)
            datarate_achieved = get_datarate(int(self.ID), N, node_list)

            # self learning
            while len(not_chosen) > 2: 
                if datarate_achieved >= self.min_datarate:
                    break

                k_max = not_chosen[np.random.randint(len(not_chosen))]
                self.BSI[k_max] = 1.0
                not_chosen.remove(k_max)

                self.publish_BSI(DB)
                datarate_achieved = get_datarate(int(self.ID), N, node_list)            

        self.publish_BSI(DB)
    

In [58]:
def get_datarates(opt, N, node_list):
    datarate = np.zeros(N, dtype='float64')
    SE = np.zeros(N, dtype='float64') # average spectral efficiency
    n_iter = 1000

    for iter in range(n_iter):
        H = np.random.exponential(1, (N, N))

    for i in range(N):
        for j in range(N):
            H[i][j] = H[j][i] 
    
    for i in range(N):
        BSI_i = node_list[i].BSI
        c = 0
        for m in range(M):
            if BSI_i[m] > 0:
                I_power = 0.0
                for j in range(N):
                    if j!=i:
                        if node_list[j].BSI[m] > 0:
                            d = distance(i, j, node_list)
                            I_power += H[i][j]*P*np.power(max(1.0,d), -alpha)

                SINR = H[i][i]*P*np.power(max(1.0,R), -alpha)/(noise + I_power)
                datarate[i] += B*np.log2(1 + SINR) 
                c += 1
                if c>0:
                    SE[i] += datarate[i]/(c*B)
    if opt:
        return datarate/n_iter
    return SE/n_iter


def get_datarate(i, N, node_list):
    datarate = 0.0
    BSI_i = node_list[i].BSI
    for m in range(M):
        if BSI_i[m] > 0:
            I_power = 0.0
            for j in range(N):
                if j!=i:
                    if node_list[j].BSI[m] > 0:
                        d = distance(i, j, node_list)
                        I_power += P*np.power(max(1.0,d), -alpha)
            SINR = P*np.power(max(1.0,R), -alpha)/(noise + I_power)
            datarate += B*np.log2(1 + SINR)

    return datarate


def get_datarates_all(opt, N, node_list):
    n_iter = N_ITER
    datarate = np.zeros((N, n_iter), dtype='float64')
    SE = np.zeros((N, n_iter), dtype='float64')

    for iter in range(n_iter):
        H = np.random.exponential(1, (N, N))

        for i in range(N):
            for j in range(N):
                H[i][j] = H[j][i] 

        for i in range(N):
            BSI_i = node_list[i].BSI
            c = 0
            for m in range(M):
                if BSI_i[m] > 0:
                    I_power = 0.0
                    for j in range(N):
                        if j!=i:
                            if node_list[j].BSI[m] > 0:
                                d = distance(i, j, node_list)
                                I_power += H[i][j]*P*np.power(max(1.0,d), -alpha)
                    SINR = H[i][i]*P*np.power(max(1.0,R), -alpha)/(noise + I_power)
                    datarate[i][iter] += B*np.log2(1 + SINR)
                    c+= 1 
                SE[i][iter] += datarate[i][iter]/(c*B)

    if opt:
        return SE

    return datarate

In [59]:
def distance(i, j, node_list):
    x_i = node_list[i].x
    y_i = node_list[i].y
    x_j = node_list[j].x
    y_j = node_list[j].y
    
    return np.sqrt( np.square(x_i-x_j) + np.square(y_i-y_j) )

In [60]:
def simulate(iter_val, density, d_max, N):
    
    area = (N/density)*1e6
    L = np.sqrt(area)
    
    DB = {}
    node_list = []
    
    X = np.random.uniform(-L/2, L/2, size=(N,1))
    Y = np.random.uniform(-L/2, L/2, size=(N,1))

    for i in range(N):
        node_list.append(Node(X[i], Y[i], i))
        
    for i in range(N):
        for j in range(i+1, N):
            d = distance(i, j, node_list)
            
            if d < d_max:
                node_list[i].add_neighbor(j, d)
                node_list[j].add_neighbor(i, d)
                
    # init BSI of all nodes
    for i in range(N):
        node_list[i].init_BSI()
        node_list[i].publish_BSI(DB)
        
    # set min datarate requirement
    for i in range(N):
        node_list[i].min_datarate = get_datarate(i, N, node_list)
        
    # create a Trigger Poisson sequence
    poisson_triggers = list(np.random.randint(N, size=turn_count*N))
    
    # greedy results
    D_greedy = get_datarates_all(0, N, node_list)
#     S_greedy = get_datarates_all(1, N, node_list)
    
    # sequential decisions
    for i in poisson_triggers:
        node_list[i].update_BSI(N, DB, node_list)
    
    # democratic results
    D_demo = get_datarates_all(0, N, node_list)
#     S_demo = get_datarates_all(1, N, node_list)
    
    
    np.savetxt("./data/synthetic/greedy/D"+ \
               '_'.join(str(a) for a in [N, int(density), int(d_max), iter_val]) + ".csv", \
               D_greedy, delimiter=",")
#     np.savetxt("./data/synthetic/greedy/S"+ \
#                '_'.join(str(a) for a in [N, int(density), int(d_max), iter_val]) + ".csv", \
#                S_greedy, delimiter=",")
    
    np.savetxt("./data/synthetic/demo/D"+ \
               '_'.join(str(a) for a in [N, int(density), int(d_max), iter_val]) + ".csv", \
               D_demo, delimiter=",")
#     np.savetxt("./data/synthetic/demo/S"+ \
#                '_'.join(str(a) for a in [N, int(density), int(d_max), iter_val]) + ".csv", \
#                S_demo, delimiter=",")
    

In [61]:
density_vec = [25.0, 125.0, 250.0, 375.0, 500.0, 625.0]
N_vec = [25, 30, 35, 40, 45, 50]
radius_vec = [50.0, 100.0, 150.0, 200.0, 250.0, 300.0]


In [52]:
N = 50
for density in density_vec:
    for radius in radius_vec:
        for iter_val in range(10):
            print(iter_val, int(density), radius, N)
            simulate(iter_val, density, radius, N)


0 25 50.0 50


/var/folders/ds/8r4fp2095114bxbwmtj9b3y40000gn/T/ipykernel_6068/4104199059.py:78: RuntimeWarning: invalid value encountered in double_scalars
  SE[i][iter] += datarate[i][iter]/(c*B)


1 25 50.0 50
2 25 50.0 50
3 25 50.0 50
4 25 50.0 50
5 25 50.0 50
6 25 50.0 50
7 25 50.0 50
8 25 50.0 50
9 25 50.0 50
0 25 100.0 50
1 25 100.0 50
2 25 100.0 50
3 25 100.0 50
4 25 100.0 50
5 25 100.0 50
6 25 100.0 50
7 25 100.0 50
8 25 100.0 50
9 25 100.0 50
0 25 150.0 50
1 25 150.0 50
2 25 150.0 50
3 25 150.0 50
4 25 150.0 50
5 25 150.0 50
6 25 150.0 50
7 25 150.0 50
8 25 150.0 50
9 25 150.0 50
0 25 200.0 50
1 25 200.0 50
2 25 200.0 50
3 25 200.0 50
4 25 200.0 50
5 25 200.0 50
6 25 200.0 50
7 25 200.0 50
8 25 200.0 50
9 25 200.0 50
0 25 250.0 50
1 25 250.0 50
2 25 250.0 50
3 25 250.0 50
4 25 250.0 50
5 25 250.0 50
6 25 250.0 50
7 25 250.0 50
8 25 250.0 50
9 25 250.0 50
0 25 300.0 50
1 25 300.0 50
2 25 300.0 50
3 25 300.0 50
4 25 300.0 50
5 25 300.0 50
6 25 300.0 50
7 25 300.0 50
8 25 300.0 50
9 25 300.0 50
0 125 50.0 50
1 125 50.0 50
2 125 50.0 50
3 125 50.0 50
4 125 50.0 50
5 125 50.0 50
6 125 50.0 50
7 125 50.0 50


In [62]:
density = 250.0
radius = 200
for N in [10, 20, 30, 40, 50]:
    for iter_val in range(10):
        print(iter_val, int(density), radius, N)
        simulate(iter_val, density, radius, N)

0 250 200 10


/var/folders/ds/8r4fp2095114bxbwmtj9b3y40000gn/T/ipykernel_6068/4104199059.py:78: RuntimeWarning: invalid value encountered in double_scalars
  SE[i][iter] += datarate[i][iter]/(c*B)


1 250 200 10
2 250 200 10
3 250 200 10
4 250 200 10
5 250 200 10
6 250 200 10
7 250 200 10
8 250 200 10
9 250 200 10
0 250 200 20
1 250 200 20
2 250 200 20
3 250 200 20
4 250 200 20
5 250 200 20
6 250 200 20
7 250 200 20
8 250 200 20
9 250 200 20
0 250 200 30
1 250 200 30
2 250 200 30
3 250 200 30
4 250 200 30
5 250 200 30
6 250 200 30
7 250 200 30
8 250 200 30
9 250 200 30
0 250 200 40
1 250 200 40
2 250 200 40
3 250 200 40
4 250 200 40
5 250 200 40
6 250 200 40
7 250 200 40
8 250 200 40
9 250 200 40
0 250 200 50
1 250 200 50
2 250 200 50
3 250 200 50
4 250 200 50
5 250 200 50
6 250 200 50
7 250 200 50
8 250 200 50
9 250 200 50
